In [43]:
#Copyright (c) 2022 Giovanni Squillero
#https://github.com/squillero/computational-intelligence
#Free for personal or classroom use; see LICENSE.md for details.

from collections import namedtuple
import random

In [44]:
N = 100

POPULATION_SIZE = N*2            
OFFSPRING_SIZE = int(N*1.5)         

NUM_GENERATIONS = 1000          

TOURNAMENT_SIZE = int(N/3)
GENETIC_OPERATOR_RANDOMNESS = 0.3

In [45]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [46]:
GOAL={i for i in range(N)}
#print(GOAL)
list_of_lists = problem(N)
tmp =  {tuple(x) for x in list_of_lists}    # remove repeated inner list
list_of_lists = list(tmp)                   
#print(list_of_lists)
PROBLEM_SIZE = len(list_of_lists)           


In [47]:
Individual = namedtuple("Individual", ["genome", "fitness"])

#convert genome into a singol list
def gen2List(genome):
    list = []
    for i, g in enumerate(genome):
        if g:
            list += list_of_lists[i]
    return list

#implementation of the tournament
def tournament(population, tournament_size=TOURNAMENT_SIZE):          
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

#implementation of 2 different types of cross over
def one_cut_cross_over(g1, g2):                      
    cut = random.randint(0, PROBLEM_SIZE)
    return g1[:cut] + g2[cut:]

def uniform_cross_over(g1, g2):
    new_genoma = []
    for i in range(PROBLEM_SIZE):
        if i%2:
            new_genoma.append(g1[i])
        else:
            new_genoma.append(g2[i])
    return tuple(new_genoma)

#implementation of mutation
def mutation(g):                                
    point = random.randint(0, PROBLEM_SIZE - 1)      
    return g[:point] + (1 - g[point],) + g[point + 1 :] 

#define the fitness function: 
# we use a fitness that consist in two terms: 
# -the first one is an evaluation of how the genome is distant from the goal
# -the second one is a term of regularization that penalizes lists that are too long
def compute_fitness(genome):                                
    list = gen2List(genome)
    return (N - len(GOAL - set(list))) - pow(len(list), 1/2)

#check if the genoma satisfies the goal
def check_goal(genoma):
    if set(gen2List(genoma)) == GOAL:
        return True
    else:
        return False

INITIAL POPULATION

In [48]:
population = list()
    
for genome in [tuple(0 for _ in range(PROBLEM_SIZE)) for _ in range(POPULATION_SIZE)]:
    genome = mutation(genome)   #the initial genome of the population are created by setting randomly only one element to 1                     
    population.append(Individual(genome, compute_fitness(genome))) 

#print(population)

EVOLUTION

In [49]:
best_fit = 0
for g in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        if random.random() < GENETIC_OPERATOR_RANDOMNESS:                         
            p = tournament(population)                  
            o = mutation(p.genome)                    
        else:                                          
            p1 = tournament(population)                 
            p2 = tournament(population)
            o = uniform_cross_over(p1.genome, p2.genome)            
        f = compute_fitness(o)                                                        
        offspring.append(Individual(o, f))                 
    population += offspring      
    population = sorted(population, key=lambda i: i[1], reverse=True)[:POPULATION_SIZE]
    
    #analize the best genoma of the actual population and update parameters
    if best_fit < population[0][1] and check_goal(population[0][0]):
        best_fit = population[0][1]
        w = len(gen2List(population[0][0]))

print(f"conv_fit {best_fit} found with w : {w} and N : {N}")

conv_fit 86.0 found with w : 196 and N : 100
